In [ ]:
"""
Anaconda環境でpystanを使うには、
    conda install -c conda-forge pystan
でインストールする必要ありそう。

そうでなければ、以下のようなエラーが出る。
    command 'x86_64-apple-darwin13.4.0-clang' failed with exit status 1

"""

In [ ]:
#####################
###  1つ目の書き方
#####################

In [3]:
# http://soonraah.hatenablog.com/entry/2014/08/23/145938


import pystan


schools_code = """
data {
    int<lower=0> J; // number of schools
    real y[J]; // estimated treatment effects
    real<lower=0> sigma[J]; // s.e. of effect estimates
}
parameters {
    real mu;
    real<lower=0> tau;
    real eta[J];
}
transformed parameters {
    real theta[J];
    for (j in 1:J)
    theta[j] <- mu + tau * eta[j];
}
model {
    eta ~ normal(0, 1);
    y ~ normal(theta, sigma);
}
"""

schools_dat = {'J': 8,
               'y': [28,  8, -3,  7, -1,  1, 18, 12],
               'sigma': [15, 10, 16, 11,  9, 11, 10, 18]}

fit = pystan.stan(model_code=schools_code, data=schools_dat,
                  iter=1000, chains=4)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_95013624776d537c3cd7cd4d641c30e0 NOW.


In [4]:
print(fit)

Inference for Stan model: anon_model_95013624776d537c3cd7cd4d641c30e0.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu          7.84    0.16   5.09  -1.94   4.33   7.72  11.28  18.21   1048    1.0
tau         6.18    0.21   5.13    0.2   2.32   5.08   8.88  18.83    583   1.01
eta[1]      0.42    0.02   0.92  -1.45  -0.18   0.46   1.03   2.18   1796    1.0
eta[2]    5.4e-3    0.02   0.85  -1.66  -0.56 2.3e-3   0.57   1.64   1805    1.0
eta[3]     -0.18    0.02   0.92  -1.95  -0.81  -0.23   0.41   1.75   1722    1.0
eta[4]   -1.3e-3    0.02   0.88  -1.86  -0.57   0.02   0.58   1.77   1964    1.0
eta[5]     -0.33    0.02   0.87  -2.08   -0.9  -0.35   0.23   1.42   1529    1.0
eta[6]     -0.23    0.02   0.89  -1.98  -0.83  -0.25   0.36   1.54   1831    1.0
eta[7]      0.33    0.02   0.88  -1.45  -0.27   0.32   0.92   2.03   1613    1.0
eta

In [8]:
"""
描画されない。
pystan に関する描画実装は今後の課題。
"""
import matplotlib.pyplot as plt
import arviz
fit.plot().show()
%matplotlib inline

/var/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#####################
###  ２つ目の書き方
#####################

In [9]:
# https://qiita.com/0NE_shoT_/items/2b41ae3e8e8f2d8809c4

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


In [10]:
def generate_sample_data(num, seed=0):
    target_list = [] # 目的変数のリスト
    feature_vector_list = [] # 説明変数（特徴量）のリスト

    feature_num = 8 # 特徴量の数
    intercept = 0.2 # 切片
    weight = [0.2, 0.3, 0.5, -0.4, 0.1, 0.2, 0.5, -0.3] # 各特徴量の重み

    np.random.seed(seed=seed)
    for i in range(num):
        feature_vector = [np.random.rand() for n in range(feature_num)] # 特徴量をランダムに生成
        noise = [np.random.normal(0, 0.1) for n in range(feature_num)] # ノイズをランダムに生成
        target = sum([intercept+feature_vector[n]*weight[n]+noise[n] for n in range(feature_num)]) # 目的変数を生成

        target_list.append(target)
        feature_vector_list.append(feature_vector)

    df = pd.DataFrame(np.c_[target_list, feature_vector_list], 
                      columns=['target', 'feature0', 'feature1', 'feature2', 
                               'feature3', 'feature4', 'feature5', 'feature6', 'feature7'])
    return df

data = generate_sample_data(num=1000, seed=0)

X = data.drop('target', axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
import pystan
import arviz

In [12]:
sample_code = """
    data {
        int<lower=0> N;
        int<lower=0> D;
        matrix[N, D] X;
        vector[N] y;
        int<lower=0> N_new;
        matrix[N_new, D] X_new;
    }
    parameters {
        real w0;
        vector[D] w;
        real<lower=0> sigma;
    }
    model {
        for (i in 1:N)
            y[i] ~ normal(w0 + dot_product(X[i], w), sigma);
    }
    generated quantities {
        vector[N_new] y_new;
        for (i in 1:N_new)
            y_new[i] = normal_rng(w0 + dot_product(X_new[i], w), sigma);
    }
"""

sample_data = {
    'N': X_train.shape[0],
    'D': X_train.shape[1],
    'X': X_train,
    'y': y_train,
    'N_new': X_test.shape[0],
    'X_new': X_test
}

In [13]:
sm = pystan.StanModel(model_code=sample_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_92cd13fa4b6e158fdf4f4ede934e6196 NOW.


In [14]:
fit = sm.sampling(data=sample_data, iter=1000, chains=4)
print(fit)

Inference for Stan model: anon_model_92cd13fa4b6e158fdf4f4ede934e6196.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
w0            1.6  1.2e-3   0.05   1.51   1.57    1.6   1.64    1.7   1559    1.0
w[1]         0.19  6.1e-4   0.03   0.13   0.17   0.19   0.21   0.25   2912    1.0
w[2]         0.35  6.0e-4   0.03   0.29   0.33   0.35   0.37   0.41   2902    1.0
w[3]          0.5  6.5e-4   0.03   0.43   0.47    0.5   0.52   0.56   2641    1.0
w[4]        -0.41  6.6e-4   0.03  -0.48  -0.44  -0.41  -0.39  -0.35   2748    1.0
w[5]         0.08  6.5e-4   0.03   0.02   0.06   0.08   0.11   0.15   2679    1.0
w[6]         0.21  6.6e-4   0.03   0.15   0.19   0.21   0.23   0.28   2469    1.0
w[7]         0.47  6.7e-4   0.03   0.41   0.45   0.47   0.49   0.53   2459    1.0
w[8]        -0.33  6.5e-4   0.03  -0.39  -0.35  -0.33  -0.31  -0.26   2665 

In [16]:
y_pred = fit['y_new'].mean(axis=0)
print('MSE(test) = {:.2f}'.format(mean_squared_error(y_test, y_pred)))
print('R^2(test) = {:.2f}'.format(r2_score(y_test, y_pred)))

MSE(test) = 0.07
R^2(test) = 0.50


In [17]:
"""
描画されない。（遅すぎて途中で止めた。）
pystan に関する描画実装は今後の課題。
"""

arviz.plot_trace(fit)
plt.show()

/var/root/anaconda3/lib/python3.7/site-packages/arviz/plots/traceplot.py:155: SyntaxWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (210), generating only 40 plots
  SyntaxWarning,


KeyboardInterrupt: 